In [ ]:
# Install required packages and set up Chrome WebDriver
!pip install selenium beautifulsoup4
!apt-get update
!apt-get -q -y install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver_manager

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,247 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,136 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,739 kB in 2s (1,475 kB/s)
Reading package li

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
def scrape_reviews(url, driver):
    driver.get(url)
    time.sleep(2)

    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'btn-group-btn') and contains(@class, 'next')]"))
        )
        next_button.click()
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException):
        return None

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    review_elements = soup.find_all("div", class_=lambda x: x and x.startswith("topic-reply"))

    return review_elements

In [ ]:
# Initialize Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # To run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

# Initialize empty lists to store the data
names = []
comments = []
dates = []

page = 1
stop = False  # Initialize the 'stop' variable

while not stop:
    url = f"https://www.pistonheads.com/gassing/topic.asp?h=0&f=23&t=2010933&i={page * 20}"

    driver.get(url)
    time.sleep(2)

    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'btn-group-btn') and contains(@class, 'next')]"))
        )
        next_button.click()
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException):
        stop = True  # Set 'stop' to True to exit the loop
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    review_elements = soup.find_all("div", class_=lambda x: x and x.startswith("topic-reply"))

    if len(review_elements) == 0:
        stop = True
        break

    for review in review_elements:
        name_element = review.find("p", class_="author js-author")
        comment_element = review.find("div", class_="phml msg-body")
        date_element = review.find("span", class_="timestamp")

        name = name_element.text.strip() if name_element else ""
        comment = comment_element.text.strip() if comment_element else ""
        date = date_element.text.strip() if date_element else ""

        names.append(name)
        comments.append(comment)
        dates.append(date)

    page += 1

In [ ]:
# Close the webdriver
driver.quit()

In [ ]:
# Create a pandas DataFrame from the scraped data
pistonheads1 = pd.DataFrame({
    "Name": names,
    "Comment": comments,
    "Date": dates
})

In [ ]:
# Display the review data
print(pistonheads1)

            Name                                            Comment  \
0        flukey5  T1berious said:\n Very interesting,I don't see...   
1        Maracus  WayOutWest said:\n\nT1berious said:\n Very int...   
2       Brett748  I don’t want an EV until I absolutely have to ...   
3          raspy  SRT77 said:\n On their website the 0 - 100km t...   
4         rewild  Every new small EV that comes out just highlig...   
..           ...                                                ...   
120     Snowbins  I too shall try to keep an open mind.  Has any...   
121  Frimley111R  I just don't see how they plan to sell any at ...   
122     Silvanus  Frimley111R said:\n I just don't see how they ...   
123     ecsrobin  Frimley111R said:\n I just don't see how they ...   
124        Hitch  I'd love to know what the real world range is....   

                            Date  
0     Tuesday 22nd November 2022  
1     Tuesday 22nd November 2022  
2     Tuesday 22nd November 2022  
3     T

In [ ]:
# Save the DataFrame to a CSV file
csv_filename = "pistonheads1.csv"  # You can choose the desired file name
pistonheads1.to_csv(csv_filename, index=False)

In [ ]:
# Display a message after saving
print(f"Scraped data saved to {csv_filename}")

Scraped data saved to pistonheads1.csv


In [ ]:
import os

In [ ]:
# Define the folder name
folder_name = "Colab_Data"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create the folder in Google Drive
drive_path = "/content/drive/MyDrive/"
folder_path = os.path.join(drive_path, folder_name)

In [ ]:
# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

print(f"Folder '{folder_name}' created in Google Drive")

Folder 'Colab_Data' created in Google Drive


In [ ]:
csv_filename = os.path.join(folder_path, "pistonheads1.csv")
review_data.to_csv(csv_filename, index=False)

print(f"CSV file saved to '{csv_filename}'")

CSV file saved to '/content/drive/MyDrive/Colab_Data/pistonheads1.csv'
